In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import Input
from keras.utils import np_utils
from keras.layers.core import Dense
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils import class_weight
import math

In [4]:
#data = pd.read_csv('../input/kddcup.data_10_percent_corrected')
#data
labels = pd.read_csv("labels.csv",sep = ":",header = None)
data   = pd.read_csv("kddcup.data_10_percent_corrected", names = labels.iloc[:,0].values)
data_onehotencoded = data.copy()

In [5]:
# categorical_index = [1,2,3,6,11,20,21]
# unique_list = []

# unique1 = data.iloc[:,1].unique()
# unique_list.append(unique1)
# unique2 = data.iloc[:,2].unique()
# unique_list.append(unique2)
# unique3 = data.iloc[:,3].unique()
# unique_list.append(unique3)
# unique4 = data.iloc[:,6].unique()
# unique_list.append(unique4)
# unique5 = data.iloc[:,11].unique()
# unique_list.append(unique5)
# unique6 = data.iloc[:,20].unique()
# unique_list.append(unique6)
# unique7 = data.iloc[:,21].unique()
# unique_list.append(unique7)
# #print(unique_list)
# #print(len(unique_list))
# #print(unique2)
lb_make = LabelEncoder()
onehot_encoder = OneHotEncoder(sparse=False)
cat_columns = labels.loc[labels.iloc[:,1] == " symbolic.",0].values
data[cat_columns] = data[cat_columns].apply(lambda col: lb_make.fit_transform(col))

In [7]:
replacements = {
        r'(smurf.|neptune.|back.|teardrop.|pod.|land.)' : 'dos',
        r'(normal.)' : 'normal',
        r'(satan.|ipsweep.|portsweep.|nmap.)' : 'probe',
        r'(warezclient.|guess_passwd.|warezmaster.|imap.|ftp_write.|multihop.|phf.|spy.)' : 'r2l',
        r'(buffer_overflow.|rootkit.|loadmodule.|perl.)' : 'u2r'
    }
data.replace(replacements, regex=True, inplace=True)
print(data.iloc[:,41].unique())

['normal' 'u2r' 'dos' 'r2l' 'probe']


In [8]:
def ffr(df, t):
    classes = ['normal', 'u2r', 'dos', 'r2l', 'probe']
    df_new = df.iloc[:,:-1]
    var_d = []
    for clas in classes:
        temp = df.loc[df['output'] == clas]
        #print("is unique ", temp.output.unique())
        temp = temp.iloc[:,:-1].values
        #print("temp = ", temp)
        min_max_scaler = preprocessing.MinMaxScaler()
        temp_scaled = min_max_scaler.fit_transform(temp)
        #print("temp_scaled = ", temp_scaled)
        mean_ind = temp_scaled.mean(axis = 0)
        #print("mean_ind = ", mean_ind)
        var_d_f = np.square(mean_ind - temp).mean(axis = 0)
        #print("var_d_f_f = ", var_d_f)
        var_d.append(var_d_f)
    var_d = np.array(var_d)
    #print(var_d)
    var_d_mean = var_d.mean(axis = 0)
    
    
#     var = np.zeros(len(mean_means))
#     for i in means:
#         var += np.square(i - mean_means)
#     var /= len(mean_means)
#     #print(var)
    indexes = list(np.argsort(var_d_mean))
    filtered_indexes = indexes[:t]
    filtered_indexes.append(41)
    filtered_data = df.iloc[:, filtered_indexes]
    return filtered_data

In [7]:
def NN(data,feature):
    epoch = 100
    batchsize = 32

    data = data.sample(frac=1,random_state=200)
    Input1 = data.iloc[:,:-1]
    Output1 = data.iloc[:,-1]
    Y = np.array(Output1)
    encoder = LabelEncoder()
    encoder.fit(Y)
    encoded_Y = encoder.transform(Y)
    print(str(encoder.classes_))
    Input1 = np.array(Input1)
    Y = np_utils.to_categorical(encoded_Y)
    print(Input1.shape)
    print(Y.shape)
    #print(Output1.shape)
    print(type(Input1))
    print(type(Output1))
    Input1 = Input1.astype(float)
    split = int(math.ceil(0.8 * len(Input1)))
    X_train = Input1[0:split]
    Y_train = Y[0:split]

    X_test = Input1[split:]
    Y_test = Y[split:]

    print(X_train.shape)
    print(X_test.shape)
    print(Y_train.shape)
    print(Y_test.shape)
    flatten_Y_train = Y_train.ravel()
    class_weights = class_weight.compute_class_weight('balanced' ,np.unique(Y_train) ,flatten_Y_train)
    
    
    inp = Input(shape=(feature,))
    hidden1 = Dense(40,activation='sigmoid')(inp)
    hidden2 = Dense(40,activation='sigmoid')(hidden1)
    out = Dense(5,activation='softmax')(hidden2)

    model = Model(inp,out)
    model.compile(optimizer='adam',loss='categorical_crossentropy')
    model.summary()

    import time

    start = time.time()
    model.fit(X_train,Y_train,epochs = epoch,batch_size=batchsize,class_weight=class_weights)
    end = time.time()
    print("Training Time for Multi-Class NN : ",(end - start))
    
    predicted = model.predict(X_test)
    #predicted = predicted.ravel()
    print(predicted.shape)
    print(predicted)
    #print(len(predicted))
    Y_predicted = np.empty((len(predicted),))
    for i in range(len(predicted)):
        Y_predicted[i] = np.argmax(predicted[i])

    print(Y_predicted)
    #y_pre,y_pred_counts = np.unique(Y_predicted,return_counts=True)
    #print(y_pre,y_pred_counts)

    # Y_predicted = Y_predicted.astype(int)


    Y_test_final = np.empty((len(Y_test),))
    for i in range(len(Y_test)):
        Y_test_final[i] = np.argmax(Y_test[i])


    print(Y_predicted.shape)
    print(Y_test_final.shape)
    print(type(Y_predicted))
    print(type(Y_test_final))

    print("Y_predicted Unique and its Frequency :")
    y_pre,y_pred_counts = np.unique(Y_predicted,return_counts=True)
    print(y_pre,y_pred_counts)

    print("Y_test Final unique and its Frequency : ")
    y_test_final,y_test_counts = np.unique(Y_test_final,return_counts=True)
    print(y_test_final,y_test_counts)

    conf = confusion_matrix(Y_test_final,Y_predicted)
    print(conf)

    precision,recall,f1,support = precision_recall_fscore_support(Y_test_final,Y_predicted)
    print("Precision : ",precision)
    print("Recall : ",recall)
    print("f1 : ",f1)
    print("Support : ",support)
    

Using TensorFlow backend.


In [ ]:
from sklearn import preprocessing
for i in range(10,31,10):
    new_dat = ffr(data, i)
    print("features = ", i)
    print(new_dat.shape)
    print(type(new_dat))
    print("With Feature size : ", i)
    NN(new_dat,i)